## Importing packages

In [3]:
import sys
sys.path.append('/home/class_NI2021/ctxctl_contrib_2023')
import samna
import samna.dynapse1 as dyn1
from dynapse1constants import *
import dynapse1utils as ut
import netgen as n
import params
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import os

## Looking for available devices

In [2]:
# Checking the list of unopened devices
devices = samna.device.get_unopened_devices()

if len(devices) == 0:
    raise Exception("no device detected!")

for i in range(len(devices)):
    print("["+str(i)+"]: ", devices[i], "serial_number", devices[i].serial_number)

[0]:  Bus 3 Device 4 Dynapse1DevKit serial_number 00000000
[1]:  Bus 1 Device 10 Dynapse1DevKit serial_number 00000031
[2]:  Bus 1 Device 15 Dynapse1DevKit serial_number 00000001


In [3]:
# Select one device from the list
model,no_gui = ut.open_dynapse1(gui=False, sender_port=17654, receiver_port=17523, select_device=True) # returns Dynapse1Model

ERROR: Address already in use, please re-run open_device()!
[0]:  Bus 3 Device 4 Dynapse1DevKit serial_number 00000000
[1]:  Bus 1 Device 10 Dynapse1DevKit serial_number 00000031
[2]:  Bus 1 Device 15 Dynapse1DevKit serial_number 00000001


In [32]:
def build_inh_OR(n_inputs, debug):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    api  = model.get_dynapse1_api()

    # ---- Parameters ----
    n_delay = 1

    # ---- Spike generator ----
    spike_generator_ids = range(1, n_inputs + 2)
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    id_ini = 1
    id_end = id_ini + n_delay
    delay_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 1
    nor_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 1
    not_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()
    total_synapses = 0

    # OP delayed
    net_gen.add_connection(spike_generators.neurons[0], delay_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    total_synapses = total_synapses + 1
    for i in range(1, n_delay):
        net_gen.add_connection(delay_pop.neurons[i - 1], delay_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)
        total_synapses = total_synapses + 1

    # OP to NOR
    net_gen.add_connection(spike_generators.neurons[0], nor_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    total_synapses = total_synapses + 1

    # OP to NOT
    net_gen.add_connection(delay_pop.neurons[-1], not_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    total_synapses = total_synapses + 1

    # Inputs to NOR
    for i in range(n_inputs):
        net_gen.add_connection(spike_generators.neurons[i + 1], nor_pop.neurons[0], dyn1.Dynapse1SynType.GABA_B)
        total_synapses = total_synapses + 1

    # NOR to NOT
    net_gen.add_connection(nor_pop.neurons[0], not_pop.neurons[0], dyn1.Dynapse1SynType.GABA_B)
    total_synapses = total_synapses + 1

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Print resources
    neurons_count = id_end - 1
    neurons_network = len(str(net_gen.network).splitlines()) - 1
    neurons_formula = 3

    synapses_count = total_synapses
    synapses_network = str(net_gen.network).count("('")
    synapses_formula = n_inputs + 4

    if debug:
        print(net_gen.network)

        print("Number of neurons (using count): " + str(neurons_count))
        print("Number of neurons (using net_gen.network): " + str(neurons_network))
        print("Number of neurons (using the formula): " + str(neurons_formula))

        print("Number of synapses (using count): " + str(synapses_count))
        print("Number of synapses (using net_gen.network): " + str(synapses_network))
        print("Number of synapses (using the formula): " + str(synapses_formula))

    if neurons_count == neurons_network and neurons_network == neurons_formula and synapses_count == synapses_network and synapses_network == synapses_formula:
        print("Test with " + str(n_inputs) + ": OK!")
    else:
        print("Test with " + str(n_inputs) + ": FAILED!")

In [33]:
for i in range(1, 51):
    build_inh_OR(i, 0)

Test with 1: OK!
New configuration applied to DYNAP-SE1!
Test with 2: OK!
New configuration applied to DYNAP-SE1!
Test with 3: OK!
New configuration applied to DYNAP-SE1!
Test with 4: OK!
New configuration applied to DYNAP-SE1!
Test with 5: OK!
New configuration applied to DYNAP-SE1!
Test with 6: OK!
New configuration applied to DYNAP-SE1!
Test with 7: OK!
New configuration applied to DYNAP-SE1!
Test with 8: OK!
New configuration applied to DYNAP-SE1!
Test with 9: OK!New configuration applied to DYNAP-SE1!

Test with 10: OK!
New configuration applied to DYNAP-SE1!
Test with 11: OK!
New configuration applied to DYNAP-SE1!
Test with 12: OK!
New configuration applied to DYNAP-SE1!
Test with 13: OK!
New configuration applied to DYNAP-SE1!
Test with 14: OK!
New configuration applied to DYNAP-SE1!
Test with 15: OK!
New configuration applied to DYNAP-SE1!
Test with 16: OK!
New configuration applied to DYNAP-SE1!
Test with 17: OK!
New configuration applied to DYNAP-SE1!
Test with 18: OK!
New c

In [63]:
def build_alt_OR(n_inputs, debug):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    api  = model.get_dynapse1_api()

    # ---- Parameters ----
    n_delay = 1

    # ---- Spike generator ----
    spike_generator_ids = range(1, n_inputs + 1)
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    id_ini = 1
    id_end = id_ini + n_inputs
    or_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 1
    join_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()
    total_synapses = 0

    # OP_i to neuron i
    for i in range(n_inputs):
        net_gen.add_connection(spike_generators.neurons[i], or_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)
        total_synapses = total_synapses + 1

        for j in range(i):
            net_gen.add_connection(spike_generators.neurons[j], or_pop.neurons[i], dyn1.Dynapse1SynType.GABA_B)
            total_synapses = total_synapses + 1

        # Joining from or_pop
        net_gen.add_connection(or_pop.neurons[i], join_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
        total_synapses = total_synapses + 1

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Print resources
    neurons_count = id_end - 1
    neurons_network = len(str(net_gen.network).splitlines()) - 1
    neurons_formula = n_inputs + 1

    synapses_count = total_synapses
    synapses_network = str(net_gen.network).count("('")
    synapses_formula = 1 / 2 * n_inputs ** 2 + 3 / 2 * n_inputs

    if debug:
        print(net_gen.network)

        print("Number of neurons (using count): " + str(neurons_count))
        print("Number of neurons (using net_gen.network): " + str(neurons_network))
        print("Number of neurons (using the formula): " + str(neurons_formula))

        print("Number of synapses (using count): " + str(synapses_count))
        print("Number of synapses (using net_gen.network): " + str(synapses_network))
        print("Number of synapses (using the formula): " + str(synapses_formula))

    if neurons_count == neurons_network and neurons_network == neurons_formula and synapses_count == synapses_network and synapses_network == synapses_formula:
        print("Test with " + str(n_inputs) + ": OK!")
    else:
        print("Test with " + str(n_inputs) + ": FAILED!")

In [64]:
for i in range(1, 51):
    build_alt_OR(i, 0)

Test with 1: OK!New configuration applied to DYNAP-SE1!

Test with 2: OK!
New configuration applied to DYNAP-SE1!
Test with 3: OK!
New configuration applied to DYNAP-SE1!
Test with 4: OK!
New configuration applied to DYNAP-SE1!
Test with 5: OK!
New configuration applied to DYNAP-SE1!
Test with 6: OK!
New configuration applied to DYNAP-SE1!
Test with 7: OK!
New configuration applied to DYNAP-SE1!
Test with 8: OK!
New configuration applied to DYNAP-SE1!
Test with 9: OK!
New configuration applied to DYNAP-SE1!
Test with 10: OK!
New configuration applied to DYNAP-SE1!
Test with 11: OK!
New configuration applied to DYNAP-SE1!
Test with 12: OK!
New configuration applied to DYNAP-SE1!
Test with 13: OK!
New configuration applied to DYNAP-SE1!
Test with 14: OK!
New configuration applied to DYNAP-SE1!
Test with 15: OK!
New configuration applied to DYNAP-SE1!
Test with 16: OK!New configuration applied to DYNAP-SE1!

Test with 17: OK!
New configuration applied to DYNAP-SE1!
Test with 18: OK!
New c

In [34]:
def build_inh_AND(n_inputs, debug):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    api  = model.get_dynapse1_api()

    # ---- Parameters ----
    n_delay = 1

    # ---- Spike generator ----
    spike_generator_ids = range(1, n_inputs + 2)
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    id_ini = 1
    id_end = id_ini + n_delay
    delay_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += n_inputs
    not_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 1
    nor_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()
    total_synapses = 0

    # OP delayed
    net_gen.add_connection(spike_generators.neurons[0], delay_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    total_synapses = total_synapses + 1
    for i in range(1, n_delay):
        net_gen.add_connection(delay_pop.neurons[i - 1], delay_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)
        total_synapses = total_synapses + 1

    # OP to NOT
    for i in range(n_inputs):
        net_gen.add_connection(spike_generators.neurons[0], not_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)
        total_synapses = total_synapses + 1

    # OP to NOR
    net_gen.add_connection(delay_pop.neurons[-1], nor_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    total_synapses = total_synapses + 1

    # Inputs to NOT
    for i in range(n_inputs):
        net_gen.add_connection(spike_generators.neurons[i + 1], not_pop.neurons[i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses = total_synapses + 1

    # NOT to NOR
    for i in range(n_inputs):
        net_gen.add_connection(not_pop.neurons[i], nor_pop.neurons[0], dyn1.Dynapse1SynType.GABA_B)
        total_synapses = total_synapses + 1

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Print resources
    neurons_count = id_end - 1
    neurons_network = len(str(net_gen.network).splitlines()) - 1
    neurons_formula = n_inputs + 2

    synapses_count = total_synapses
    synapses_network = str(net_gen.network).count("('")
    synapses_formula = 3 * n_inputs + 2

    if debug:
        print(net_gen.network)

        print("Number of neurons (using count): " + str(neurons_count))
        print("Number of neurons (using net_gen.network): " + str(neurons_network))
        print("Number of neurons (using the formula): " + str(neurons_formula))

        print("Number of synapses (using count): " + str(synapses_count))
        print("Number of synapses (using net_gen.network): " + str(synapses_network))
        print("Number of synapses (using the formula): " + str(synapses_formula))

    if neurons_count == neurons_network and neurons_network == neurons_formula and synapses_count == synapses_network and synapses_network == synapses_formula:
        print("Test with " + str(n_inputs) + ": OK!")
    else:
        print("Test with " + str(n_inputs) + ": FAILED!")

In [36]:
for i in range(1, 51):
    build_inh_AND(i, 0)

Test with 1: OK!
New configuration applied to DYNAP-SE1!
Test with 2: OK!
New configuration applied to DYNAP-SE1!
Test with 3: OK!
New configuration applied to DYNAP-SE1!
Test with 4: OK!
New configuration applied to DYNAP-SE1!
Test with 5: OK!New configuration applied to DYNAP-SE1!

Test with 6: OK!
New configuration applied to DYNAP-SE1!
Test with 7: OK!New configuration applied to DYNAP-SE1!

Test with 8: OK!
New configuration applied to DYNAP-SE1!
Test with 9: OK!New configuration applied to DYNAP-SE1!

Test with 10: OK!
New configuration applied to DYNAP-SE1!
Test with 11: OK!
New configuration applied to DYNAP-SE1!
Test with 12: OK!
New configuration applied to DYNAP-SE1!
Test with 13: OK!
New configuration applied to DYNAP-SE1!
Test with 14: OK!
New configuration applied to DYNAP-SE1!
Test with 15: OK!
New configuration applied to DYNAP-SE1!
Test with 16: OK!
New configuration applied to DYNAP-SE1!
Test with 17: OK!
New configuration applied to DYNAP-SE1!
Test with 18: OK!
New c

In [4]:
def build_adder(n_bits, debug):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    api  = model.get_dynapse1_api()

    # ---- Spike generator ----
    id_ini = 1
    id_end = id_ini + n_bits * 2 + 1
    spike_generator_ids = np.array(range(id_ini, id_end))
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    core_id = 1

    id_ini = id_end
    if n_bits > 1:
        id_end = id_ini + 5 * (n_bits - 1)
    else:
        id_end = id_ini + 1
    op_delay_pop = n.NeuronGroup(0, core_id, range(id_ini, id_end), False)

    a_delay_pop = []
    for i in range(n_bits - 1):
        id_ini = id_end
        id_end += 2 + 5 * i
        a_delay_pop.append(n.NeuronGroup(0, core_id, range(id_ini, id_end), False))

    b_delay_pop = []
    for i in range(n_bits - 1):
        id_ini = id_end
        id_end += 2 + 5 * i
        b_delay_pop.append(n.NeuronGroup(0, core_id, range(id_ini, id_end), False))

    id_ini = id_end
    id_end += 6
    ha_pop = n.NeuronGroup(0, core_id, range(id_ini, id_end), False)

    if n_bits > 1:
        id_ini = id_end
        id_end += 18 * (n_bits - 1)
        fa_pop = n.NeuronGroup(0, core_id, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()
    total_synapses = 0

    # OP chain
    net_gen.add_connection(spike_generators.neurons[0], op_delay_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    total_synapses += 1
    for i in range(1, len(op_delay_pop.neurons)):
        net_gen.add_connection(op_delay_pop.neurons[i - 1], op_delay_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 1

    # OP to NOT
    net_gen.add_connection(spike_generators.neurons[0], ha_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(spike_generators.neurons[0], ha_pop.neurons[1], dyn1.Dynapse1SynType.AMPA)
    total_synapses += 2
    for i in range(n_bits - 1):
        net_gen.add_connection(op_delay_pop.neurons[1 + 5 * i], fa_pop.neurons[18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(op_delay_pop.neurons[1 + 5 * i], fa_pop.neurons[18 * i + 1], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2
        
    # A and B chains -> A0 y B0 are not delayed!
    for i in range(1, n_bits):
        net_gen.add_connection(spike_generators.neurons[1 + i], a_delay_pop[i-1].neurons[0], dyn1.Dynapse1SynType.AMPA)  # A
        net_gen.add_connection(spike_generators.neurons[1 + i + n_bits], b_delay_pop[i-1].neurons[0], dyn1.Dynapse1SynType.AMPA)  # B
        total_synapses += 2

    for i in range(n_bits - 1):
        for j in range(1, len(a_delay_pop[i].neurons)):
            net_gen.add_connection(a_delay_pop[i].neurons[j - 1], a_delay_pop[i].neurons[j], dyn1.Dynapse1SynType.AMPA)
            total_synapses += 1
        for j in range(1, len(b_delay_pop[i].neurons)):
            net_gen.add_connection(b_delay_pop[i].neurons[j - 1], b_delay_pop[i].neurons[j], dyn1.Dynapse1SynType.AMPA)
            total_synapses += 1

    # Input to NOT
    net_gen.add_connection(spike_generators.neurons[1], ha_pop.neurons[0], dyn1.Dynapse1SynType.GABA_B)  # A
    net_gen.add_connection(spike_generators.neurons[1 + n_bits], ha_pop.neurons[1], dyn1.Dynapse1SynType.GABA_B)  # B
    total_synapses += 2
    for i in range(n_bits - 1):
        net_gen.add_connection(a_delay_pop[i].neurons[len(a_delay_pop[i].neurons) - 1], fa_pop.neurons[18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(b_delay_pop[i].neurons[len(b_delay_pop[i].neurons) - 1], fa_pop.neurons[18 * i + 1], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 2

    # --------- HA internal structure ----------
    # 2
    net_gen.add_connection(op_delay_pop.neurons[0], ha_pop.neurons[2], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(ha_pop.neurons[0], ha_pop.neurons[2], dyn1.Dynapse1SynType.GABA_B)
    net_gen.add_connection(ha_pop.neurons[1], ha_pop.neurons[2], dyn1.Dynapse1SynType.GABA_B)
    total_synapses += 3

    # 3
    net_gen.add_connection(ha_pop.neurons[0], ha_pop.neurons[3], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(ha_pop.neurons[1], ha_pop.neurons[3], dyn1.Dynapse1SynType.GABA_B)
    total_synapses += 2

    # 4
    net_gen.add_connection(ha_pop.neurons[0], ha_pop.neurons[4], dyn1.Dynapse1SynType.GABA_B)
    net_gen.add_connection(ha_pop.neurons[1], ha_pop.neurons[4], dyn1.Dynapse1SynType.AMPA)
    total_synapses += 2

    # 5
    net_gen.add_connection(ha_pop.neurons[3], ha_pop.neurons[5], dyn1.Dynapse1SynType.AMPA)
    net_gen.add_connection(ha_pop.neurons[4], ha_pop.neurons[5], dyn1.Dynapse1SynType.AMPA)
    total_synapses += 2

    # Connecting to the next adder
    if n_bits > 1:
        net_gen.add_connection(ha_pop.neurons[2], fa_pop.neurons[2], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 1

    # --------- FA internal structure ----------
    for i in range(n_bits - 1):
        # 3
        net_gen.add_connection(op_delay_pop.neurons[2 + 5 * i], fa_pop.neurons[3 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[0 + 18 * i], fa_pop.neurons[3 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[1 + 18 * i], fa_pop.neurons[3 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 3

        # 4
        net_gen.add_connection(fa_pop.neurons[0 + 18 * i], fa_pop.neurons[4 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[1 + 18 * i], fa_pop.neurons[4 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 2

        # 5
        net_gen.add_connection(fa_pop.neurons[0 + 18 * i], fa_pop.neurons[5 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[1 + 18 * i], fa_pop.neurons[5 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2

        # 6
        net_gen.add_connection(fa_pop.neurons[2 + 18 * i], fa_pop.neurons[6 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 1

        # 7
        net_gen.add_connection(fa_pop.neurons[3 + 18 * i], fa_pop.neurons[7 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 1

        # 8
        net_gen.add_connection(op_delay_pop.neurons[3 + 5 * i], fa_pop.neurons[8 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[4 + 18 * i], fa_pop.neurons[8 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[5 + 18 * i], fa_pop.neurons[8 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 3

        # 9
        net_gen.add_connection(fa_pop.neurons[4 + 18 * i], fa_pop.neurons[9 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[5 + 18 * i], fa_pop.neurons[9 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2

        # 10
        net_gen.add_connection(fa_pop.neurons[6 + 18 * i], fa_pop.neurons[10 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 1

        # 11
        net_gen.add_connection(op_delay_pop.neurons[3 + 5 * i], fa_pop.neurons[11 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[6 + 18 * i], fa_pop.neurons[11 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 2

        # 12
        net_gen.add_connection(fa_pop.neurons[7 + 18 * i], fa_pop.neurons[12 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 1

        # 13
        net_gen.add_connection(op_delay_pop.neurons[4 + 5 * i], fa_pop.neurons[13 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[8 + 18 * i], fa_pop.neurons[13 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[11 + 18 * i], fa_pop.neurons[13 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 3

        # 14
        net_gen.add_connection(fa_pop.neurons[9 + 18 * i], fa_pop.neurons[14 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[10 + 18 * i], fa_pop.neurons[14 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 2

        # 15
        net_gen.add_connection(fa_pop.neurons[9 + 18 * i], fa_pop.neurons[15 + 18 * i], dyn1.Dynapse1SynType.GABA_B)
        net_gen.add_connection(fa_pop.neurons[10 + 18 * i], fa_pop.neurons[15 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2

        # 16
        net_gen.add_connection(fa_pop.neurons[12 + 18 * i], fa_pop.neurons[16 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[13 + 18 * i], fa_pop.neurons[16 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2
        # Connecting to the next adder
        if i != (n_bits - 2):
            net_gen.add_connection(fa_pop.neurons[16 + 18 * i], fa_pop.neurons[2 + 18 * (i + 1)], dyn1.Dynapse1SynType.AMPA)
            total_synapses += 1

        # 17
        net_gen.add_connection(fa_pop.neurons[14 + 18 * i], fa_pop.neurons[17 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(fa_pop.neurons[15 + 18 * i], fa_pop.neurons[17 + 18 * i], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Print resources
    neurons_count = id_end - 1 - (n_bits * 2 + 1)
    neurons_network = len(str(net_gen.network).splitlines()) - 1
    neurons_formula = 5 * (n_bits - 2) ** 2 + 5 * (n_bits - 2) + 27 * (n_bits - 1) + 6

    synapses_count = total_synapses
    synapses_network = str(net_gen.network).count("('")
    synapses_formula = 5 * (n_bits - 2) ** 2 + 5 * (n_bits - 2) + 43 * (n_bits - 1) + 13

    if debug:
        print(net_gen.network)

        print("Number of neurons (using count): " + str(neurons_count))
        print("Number of neurons (using net_gen.network): " + str(neurons_network))
        print("Number of neurons (using the formula): " + str(neurons_formula))

        print("Number of synapses (using count): " + str(synapses_count))
        print("Number of synapses (using net_gen.network): " + str(synapses_network))
        print("Number of synapses (using the formula): " + str(synapses_formula))

    if neurons_count == neurons_network and neurons_network == neurons_formula and synapses_count == synapses_network and synapses_network == synapses_formula:
        print("Test with " + str(n_bits) + ": OK!")
    else:
        print("Test with " + str(n_bits) + ": FAILED!")

In [5]:
for i in range(2, 7):
    build_adder(i, 1)

Post neuron (ChipId,coreId,neuronId): incoming connections [(preNeuron,synapseType), ...]
C0c1n6: [('C0c0s1', 'AMPA')]
C0c1n7: [('C0c1n6', 'AMPA')]
C0c1n8: [('C0c1n7', 'AMPA')]
C0c1n9: [('C0c1n8', 'AMPA')]
C0c1n10: [('C0c1n9', 'AMPA')]
C0c1n15: [('C0c0s1', 'AMPA'), ('C0c0s2', 'GABA_B')]
C0c1n16: [('C0c0s1', 'AMPA'), ('C0c0s4', 'GABA_B')]
C0c1n21: [('C0c1n7', 'AMPA'), ('C0c1n12', 'GABA_B')]
C0c1n22: [('C0c1n7', 'AMPA'), ('C0c1n14', 'GABA_B')]
C0c1n11: [('C0c0s3', 'AMPA')]
C0c1n13: [('C0c0s5', 'AMPA')]
C0c1n12: [('C0c1n11', 'AMPA')]
C0c1n14: [('C0c1n13', 'AMPA')]
C0c1n17: [('C0c1n6', 'AMPA'), ('C0c1n15', 'GABA_B'), ('C0c1n16', 'GABA_B')]
C0c1n18: [('C0c1n15', 'AMPA'), ('C0c1n16', 'GABA_B')]
C0c1n19: [('C0c1n15', 'GABA_B'), ('C0c1n16', 'AMPA')]
C0c1n20: [('C0c1n18', 'AMPA'), ('C0c1n19', 'AMPA')]
C0c1n23: [('C0c1n17', 'AMPA')]
C0c1n24: [('C0c1n8', 'AMPA'), ('C0c1n21', 'GABA_B'), ('C0c1n22', 'GABA_B')]
C0c1n25: [('C0c1n21', 'AMPA'), ('C0c1n22', 'GABA_B')]
C0c1n26: [('C0c1n21', 'GABA_B'), ('

In [60]:
def build_decoder(n_bits, debug):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    api  = model.get_dynapse1_api()

    # ---- Parameters ----
    n_delay = 1

    # ---- Spike generator ----
    spike_generator_ids = range(1, n_bits + 1)
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    id_ini = 1
    id_end = id_ini + n_bits * 2
    input_layer = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += n_bits * 2 + 1
    middle_layer = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 2 ** n_bits - 1
    output_layer = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()
    total_synapses = 0
        
    # Input layer
    for i in range(n_bits): # 2 * n
        net_gen.add_connection(spike_generators.neurons[i], input_layer.neurons[2 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(spike_generators.neurons[i], input_layer.neurons[2 * i + 1], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2
            
        for j in range(i + 1, n_bits):  # (n - 1) * (1 + (n - 1)) / 2 = (n ^ 2 - n) / 2
            net_gen.add_connection(spike_generators.neurons[i], input_layer.neurons[2 * j + 1], dyn1.Dynapse1SynType.GABA_B)
            total_synapses += 1

    # Middle layer
    for i in range(n_bits): # 2 * n
        net_gen.add_connection(input_layer.neurons[2 * i], middle_layer.neurons[2 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(input_layer.neurons[2 * i], middle_layer.neurons[2 * i + 1], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 2
    
        # OP to NOT -> n ^ 2
        for j in range(n_bits):   
            net_gen.add_connection(input_layer.neurons[2 * i + 1], middle_layer.neurons[2 * j + 1], dyn1.Dynapse1SynType.AMPA)
            total_synapses += 1
    
        # OP (+1) -> n
        net_gen.add_connection(input_layer.neurons[2 * i + 1], middle_layer.neurons[-1], dyn1.Dynapse1SynType.AMPA)  
        total_synapses += 1

    # Output layer 
    for i in range(1, 2 ** n_bits): # 2 ^ n - 1
        i_bin = format(i, "0" + str(n_bits) + "b").reversed()

        # Inhibition
        for j in range(len(i_bin)):  
            if i_bin[j] == '1':
                net_gen.add_connection(middle_layer.neurons[2 * j + 1], output_layer.neurons[i - 1], dyn1.Dynapse1SynType.GABA_B)  
            else:
                net_gen.add_connection(middle_layer.neurons[2 * j], output_layer.neurons[i - 1], dyn1.Dynapse1SynType.GABA_B) 
            total_synapses += 1
    
        # Excitation (OP (+1)) 
        net_gen.add_connection(middle_layer.neurons[-1], output_layer.neurons[i - 1], dyn1.Dynapse1SynType.AMPA)  
        total_synapses += 1

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Print resources
    neurons_count = id_end - 1
    neurons_network = len(str(net_gen.network).splitlines()) - 1
    neurons_formula = 2 ** n_bits + 4 * n_bits

    synapses_count = total_synapses
    synapses_network = str(net_gen.network).count("('")
    synapses_formula = (n_bits + 1) * 2 ** n_bits + 2 * n_bits ** 2 + 3 * n_bits - 1

    if debug:
        print(net_gen.network)

        print("Number of neurons (using count): " + str(neurons_count))
        print("Number of neurons (using net_gen.network): " + str(neurons_network))
        print("Number of neurons (using the formula): " + str(neurons_formula))

        print("Number of synapses (using count): " + str(synapses_count))
        print("Number of synapses (using net_gen.network): " + str(synapses_network))
        print("Number of synapses (using the formula): " + str(synapses_formula))

    if neurons_count == neurons_network and neurons_network == neurons_formula and synapses_count == synapses_network and synapses_network == synapses_formula:
        print("Test with " + str(n_bits) + ": OK!")
    else:
        print("Test with " + str(n_bits) + ": FAILED!")

In [62]:
for i in range(1, 8):
    build_decoder(i, 0)

New configuration applied to DYNAP-SE1!
Test with 1: OK!
Test with 2: OK!
New configuration applied to DYNAP-SE1!
Test with 3: OK!New configuration applied to DYNAP-SE1!

New configuration applied to DYNAP-SE1!
Test with 4: OK!
Test with 5: OK!New configuration applied to DYNAP-SE1!

Test with 6: OK!New configuration applied to DYNAP-SE1!

New configuration applied to DYNAP-SE1!
Test with 7: OK!


In [26]:
def build_decoder_no_DynapSE(n_bits, debug):
    # ---- Spike generator ----
    spike_generator_ids = range(1, n_bits + 1)
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    id_ini = 1
    id_end = id_ini + n_bits * 2
    input_layer = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += n_bits * 2 + 1
    middle_layer = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 2 ** n_bits - 1
    output_layer = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()
    total_synapses = 0
        
    # Input layer
    for i in range(n_bits): # 2 * n
        net_gen.add_connection(spike_generators.neurons[i], input_layer.neurons[2 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(spike_generators.neurons[i], input_layer.neurons[2 * i + 1], dyn1.Dynapse1SynType.AMPA)
        total_synapses += 2
            
        for j in range(i + 1, n_bits):  # (n - 1) * (1 + (n - 1)) / 2 = (n ^ 2 - n) / 2
            net_gen.add_connection(spike_generators.neurons[i], input_layer.neurons[2 * j + 1], dyn1.Dynapse1SynType.GABA_B)
            total_synapses += 1

    # Middle layer
    for i in range(n_bits): # 2 * n
        net_gen.add_connection(input_layer.neurons[2 * i], middle_layer.neurons[2 * i], dyn1.Dynapse1SynType.AMPA)
        net_gen.add_connection(input_layer.neurons[2 * i], middle_layer.neurons[2 * i + 1], dyn1.Dynapse1SynType.GABA_B)
        total_synapses += 2
    
        # OP to NOT -> n ^ 2
        for j in range(n_bits):   
            net_gen.add_connection(input_layer.neurons[2 * i + 1], middle_layer.neurons[2 * j + 1], dyn1.Dynapse1SynType.AMPA)
            total_synapses += 1
    
        # OP (+1) -> n
        net_gen.add_connection(input_layer.neurons[2 * i + 1], middle_layer.neurons[-1], dyn1.Dynapse1SynType.AMPA)  
        total_synapses += 1

    # Output layer 
    for i in range(1, 2 ** n_bits): # 2 ^ n - 1
        i_bin = format(i, "0" + str(n_bits) + "b")[::-1]

        # Inhibition
        for j in range(len(i_bin)):  
            if i_bin[j] == '1':
                net_gen.add_connection(middle_layer.neurons[2 * j + 1], output_layer.neurons[i - 1], dyn1.Dynapse1SynType.GABA_B)  
            else:
                net_gen.add_connection(middle_layer.neurons[2 * j], output_layer.neurons[i - 1], dyn1.Dynapse1SynType.GABA_B) 
            total_synapses += 1
    
        # Excitation (OP (+1)) 
        net_gen.add_connection(middle_layer.neurons[-1], output_layer.neurons[i - 1], dyn1.Dynapse1SynType.AMPA)  
        total_synapses += 1

    # Print resources
    neurons_count = id_end - 1
    neurons_network = len(str(net_gen.network).splitlines()) - 1
    neurons_formula = 2 ** n_bits + 4 * n_bits

    synapses_count = total_synapses
    synapses_network = str(net_gen.network).count("('")
    synapses_formula = (n_bits + 1) * 2 ** n_bits + 3 / 2 * n_bits ** 2 + 7 / 2 * n_bits - 1 


    if debug:
        print(net_gen.network)

        print("Number of neurons (using count): " + str(neurons_count))
        print("Number of neurons (using net_gen.network): " + str(neurons_network))
        print("Number of neurons (using the formula): " + str(neurons_formula))

        print("Number of synapses (using count): " + str(synapses_count))
        print("Number of synapses (using net_gen.network): " + str(synapses_network))
        print("Number of synapses (using the formula): " + str(synapses_formula))

    if neurons_count == neurons_network and neurons_network == neurons_formula and synapses_count == synapses_network and synapses_network == synapses_formula:
        print("Test with " + str(n_bits) + ": OK!")
    else:
        print("Test with " + str(n_bits) + ": FAILED!")

In [27]:
for i in range(1, 8):
    build_decoder_no_DynapSE(i, 1)

Post neuron (ChipId,coreId,neuronId): incoming connections [(preNeuron,synapseType), ...]
C0c1n1: [('C0c0s1', 'AMPA')]
C0c1n2: [('C0c0s1', 'AMPA')]
C0c1n3: [('C0c1n1', 'AMPA')]
C0c1n4: [('C0c1n1', 'GABA_B'), ('C0c1n2', 'AMPA')]
C0c1n5: [('C0c1n2', 'AMPA')]
C0c1n6: [('C0c1n4', 'GABA_B'), ('C0c1n5', 'AMPA')]

Number of neurons (using count): 6
Number of neurons (using net_gen.network): 6
Number of neurons (using the formula): 6
Number of synapses (using count): 8
Number of synapses (using net_gen.network): 8
Number of synapses (using the formula): 8.0
Test with 1: OK!
Post neuron (ChipId,coreId,neuronId): incoming connections [(preNeuron,synapseType), ...]
C0c1n1: [('C0c0s1', 'AMPA')]
C0c1n2: [('C0c0s1', 'AMPA')]
C0c1n4: [('C0c0s1', 'GABA_B'), ('C0c0s2', 'AMPA')]
C0c1n3: [('C0c0s2', 'AMPA')]
C0c1n5: [('C0c1n1', 'AMPA')]
C0c1n6: [('C0c1n1', 'GABA_B'), ('C0c1n2', 'AMPA'), ('C0c1n4', 'AMPA')]
C0c1n8: [('C0c1n2', 'AMPA'), ('C0c1n3', 'GABA_B'), ('C0c1n4', 'AMPA')]
C0c1n9: [('C0c1n2', 'AMPA'),

## Close the device

In [65]:
# remeber to close the device
samna.device.close_device(model)